### Setup aus MetEngSim

In [1]:
import sys # loading commands to control/navigate within the system architecture
# Loading pandas, a library for data manipulation
from os.path import join
# import xlrd
import pandas as pd
# import lxml

# Loading numpy, a library fo manipulation of numbers
import numpy as np

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
# from cobra.io import read_sbml_model, write_sbml_model, load_matlab_model

# loading escher for metabolic network visualization
# import escher
# from escher import Builder
# from time import sleep
# escher.rc['never_ask_before_quit'] = True
# list of available maps
# print(escher.list_available_maps())

# loading Memote, quality assessment of GSMM
# from memote import test_model, snapshot_report

from biolabsim import Host, Strain, Ecol

print('System ready')

System ready


In [2]:
wtHost = Ecol()
wtHost.show_BiotechSetting()
print('Wild type growth rate: {:.2f}'.format(wtHost.strain.objective))
print()
myHost = wtHost.clone_with_mutation('Mut1')
myHost.show_BiotechSetting()
print('Mutation growth rate: {:.2f}'.format(myHost.strain.objective))

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Host: Ecol
Resources: 40
Substrate: None
Strain: WT
Wild type growth rate: 0.87



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.22.1 when using version 0.23.2. This might lead to break

resetting boundaries
Host: Ecol
Resources: 40
Substrate: None
Strain: Mut1
Mutation growth rate: 0.87


### Ändern des Dataframes
#### Hinzufügen des Transkriptionsfaktors

In [4]:
def add_TranscriptionFactor(df):
    newdf = df
    
    # add TF row:
    TF = {
    'RctID':['TF'], 
    'Expression':[2], # noch zu ändern mit Help_PromoterStrength()?
    'Promoter':['GCCCATTGACCATACCGGAAGGAGATAATAAAGTTGCACG'],  # noch zu ändern (echt, random oder Fkt?)
    'ORF': ['ATGGAGATGAAGTAA'], # noch zu ändern (echt, random oder Fkt?)
    'Fluxes': [9], # noch zu ändern mit Help_FluxCalculator()?
    'Expr2Flux': [4.5] # noch zu ändern mit Help_Expr2Flux()?
    }
    dfTF = pd.DataFrame(TF)
    newdf = newdf.append(dfTF, ignore_index=True)
    
    # add TF_regulated column:
    zeroarray = np.zeros(len(newdf),dtype=int) # später zu 1 oder 0 ändern (random oder festgesetzt?)
    newdf = newdf.assign(TF_regulated=zeroarray)
    # statt 1 'TF' (RctID)
    
    # set TF as always TF_regulated:
    filt = (newdf['RctID']=='TF') # access only TF (da index sich ändern könnte, der etwas kompliziertere Weg)
    newdf.loc[filt, 'TF_regulated'] = 1 # TF immer TF_regulated

    return newdf

In [5]:
newdf = wtHost.strain.genes_df
newdf = add_TranscriptionFactor(newdf)
newdf

,RctID,Expression,Promoter,ORF,Fluxes,Expr2Flux,TF_regulated
0,PFK,1.441,GCCCATTGATAAGATAGTCAGCTCAAGGTAGTGTTGCACG,ATGGGAACAAATTGA,7.477382,5.189023,0
1,PFL,1.385,GCCCATTGATTCCCCACGAAGGCTTCTCTATTGTTGCACG,ATGCCAGTGTGCTAA,0.000000,0.000000,0
2,PGI,1.123,GCCCATTGATCGCCTCACGACGCCGCGTTAAGTTTGCACG,ATGTTTATTGAATAA,4.860861,4.328461,0
3,PGK,0.813,GCCCATTGATTTAAACGGATTACTAAAATAGGCTTGCACG,ATGTATCAAATTTAA,-16.023526,-19.709134,0
4,PGL,1.457,GCCCATTGAGTTAGGGCGGTGGGATGGGTATTTTTGCACG,ATGATTAGCTTCTAA,4.959985,3.404245,0
...,...,...,...,...,...,...,...
91,NADTRHD,2.126,GCCCATTGACTATGAACCCAGAAACCCCTAGCCTTGCACG,ATGTACAATGAATAA,0.000000,0.000000,0
92,NH4t,2.358,GCCCATTGACATTGCTGTCTAGATAGTGTAGTTTTGCACG,ATGTGTTCGAACTAA,4.765319,2.020916,0
93,O2t,3.341,GCCCATTGACAGTCAACTCGGCTCTCTTTAAACTTGCACG,ATGCCTCCCAGCTGA,21.799493,6.524841,0
94,PDH,1.756,GCCCATTGAAGCTCAACTTGTGCCGGGCTATTCTTGCACG,ATGGGTCGCGCTTAA,9.282533,5.286180,0


In [6]:
# unnötig but nett to know: random alle Gene durch TF regulieren lassen oder nicht

# import random

# rand_onoff = random.randint(0,1)
    
# newdf['TF_regulated'] = newdf['TF_regulated'].apply(lambda x: x * 0 + onoff)

# newdf

### Regulationsgleichung

In [7]:
# myEquations=['linear','Hill']

myModel.reactions[Indx].lower_bound = FluxEquation # (equationtype = myEquations[RctNewDF.loc[Indx, 'Expr2Flux']])

EquationParameter: 'TF_regulated', cTF, 'Expression'

def FluxEquation(index, WTdf, MTdf): 
    '''gibt abhängig davon, ob das Protein vom TF reguliert wird oder nicht, die Gleichung an, mit der der Flux (nur von MTdf???) berechnet wird'''

    # if/else von WT oder MT?
    
    # linear          
    if df[index,'TF_regulated'] == 0: #aus WT oder MT?
        
        return #was?
    
    # Hill
    elif df[index,'TF_regulated'] == 'TF':

        filt = (newdf['RctID']=='TF')
        
        Expr = MTdf.loc[index, 'Expression'].values
        cTF = MTdf.loc[filt, 'Expression'].values # abhängig von geändertem TF-Promotor
        K = WTdf.loc[index, 'Expression'].values # physiologischer Wert von c, Expression/Expr2Flux
        n = 1.95 # from paper                
        
        Corr_ExprFlux = (2 * Expr) / (1+(K/cTF)**n)
        
        return #was?

SyntaxError: invalid syntax (<ipython-input-7-155daad20ded>, line 5)

### Vergleich von WT und MT aus MetEngSim

In [8]:
wtHost.strain.genes_df

,RctID,Expression,Promoter,ORF,Fluxes,Expr2Flux
0,PFK,1.441,GCCCATTGATAAGATAGTCAGCTCAAGGTAGTGTTGCACG,ATGGGAACAAATTGA,7.477382,5.189023
1,PFL,1.385,GCCCATTGATTCCCCACGAAGGCTTCTCTATTGTTGCACG,ATGCCAGTGTGCTAA,0.000000,0.000000
2,PGI,1.123,GCCCATTGATCGCCTCACGACGCCGCGTTAAGTTTGCACG,ATGTTTATTGAATAA,4.860861,4.328461
3,PGK,0.813,GCCCATTGATTTAAACGGATTACTAAAATAGGCTTGCACG,ATGTATCAAATTTAA,-16.023526,-19.709134
4,PGL,1.457,GCCCATTGAGTTAGGGCGGTGGGATGGGTATTTTTGCACG,ATGATTAGCTTCTAA,4.959985,3.404245
...,...,...,...,...,...,...
90,NADH16,2.394,GCCCATTGATAGCATATGTTACGGTACGTACTCTTGCACG,ATGACCGGTGTCTGA,38.534610,16.096328
91,NADTRHD,2.126,GCCCATTGACTATGAACCCAGAAACCCCTAGCCTTGCACG,ATGTACAATGAATAA,0.000000,0.000000
92,NH4t,2.358,GCCCATTGACATTGCTGTCTAGATAGTGTAGTTTTGCACG,ATGTGTTCGAACTAA,4.765319,2.020916
93,O2t,3.341,GCCCATTGACAGTCAACTCGGCTCTCTTTAAACTTGCACG,ATGCCTCCCAGCTGA,21.799493,6.524841


In [9]:
myHost.strain.genes_df

,RctID,Expression,Promoter,ORF,Fluxes,Expr2Flux
0,PFK,1.441,GCCCATTGATAAGATAGTCAGCTCAAGGTAGTGTTGCACG,ATGGGAACAAATTGA,7.438117,5.189023
1,PFL,1.385,GCCCATTGATTCCCCACGAAGGCTTCTCTATTGTTGCACG,ATGCCAGTGTGCTAA,0.000000,0.000000
2,PGI,1.123,GCCCATTGATCGCCTCACGACGCCGCGTTAAGTTTGCACG,ATGTTTATTGAATAA,4.804540,4.328461
3,PGK,0.813,GCCCATTGATTTAAACGGATTACTAAAATAGGCTTGCACG,ATGTATCAAATTTAA,-15.954438,-19.709134
4,PGL,1.457,GCCCATTGAGTTAGGGCGGTGGGATGGGTATTTTTGCACG,ATGATTAGCTTCTAA,4.981618,3.404245
...,...,...,...,...,...,...
90,NADH16,2.394,GCCCATTGATAGCATATGTTACGGTACGTACTCTTGCACG,ATGACCGGTGTCTGA,38.429009,16.096328
91,NADTRHD,2.126,GCCCATTGACTATGAACCCAGAAACCCCTAGCCTTGCACG,ATGTACAATGAATAA,0.073198,0.000000
92,NH4t,2.349,GCCCATTGACATTGCTGTCTAGATAGTGTTGTATTGCACG,ATGTGTTCGAACTAA,4.747131,2.020916
93,O2t,3.341,GCCCATTGACAGTCAACTCGGCTCTCTTTAAACTTGCACG,ATGCCTCCCAGCTGA,21.733063,6.524841


In [10]:
mylist = np.array(wtHost.strain.genes_df['Expression'])
mylist2 = np.array(myHost.strain.genes_df['Expression'])
print('Gene expression differences')
np.round(mylist2/mylist,2)

Gene expression differences


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [11]:
myl = np.array(wtHost.strain.genes_df['Expr2Flux'])
myl2 = np.array(myHost.strain.genes_df['Expr2Flux'])
# myHost.strain.genes_df
print('Exp2Flux diff')
np.round(myl2/myl,2)

Exp2Flux diff


<ipython-input-11-793ddc7a46d2>:5: RuntimeWarning: invalid value encountered in true_divide
  np.round(myl2/myl,2)


array([ 1., nan,  1.,  1.,  1., nan, nan,  1.,  1., nan, nan, nan,  1.,
        1.,  1.,  1., nan, nan, nan, nan,  1.,  1., nan,  1.,  1., nan,
        1.,  1.,  1.,  1., nan,  1., nan,  1., nan, nan,  1.,  1.,  1.,
       nan,  1.,  1.,  1., nan, nan, nan,  1., nan, nan, nan, nan,  1.,
       nan, nan,  1.,  1., nan, nan,  1.,  1.,  1., nan, nan,  1., nan,
       nan, nan, nan, nan,  1., nan,  1.,  1.,  1.,  1., nan,  1., nan,
       nan, nan,  1.,  1.,  1., nan, nan, nan, nan,  1., nan, nan,  1.,
       nan,  1.,  1.,  1.])

In [12]:
FluxDiff = myHost.strain.genes_df['Fluxes']/wtHost.strain.genes_df['Fluxes']
print('Flux reaction differences')
np.round(np.nan_to_num(FluxDiff, nan=1, posinf=1, neginf=1),2)

Flux reaction differences


array([0.99, 1.  , 0.99, 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 0.99, 0.99, 1.  , 1.  , 1.  , 1.  , 1.  , 0.99, 1.  ,
       1.  , 0.99, 1.  , 1.  , 1.  , 1.01, 0.99, 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 0.99, 0.99, 1.01, 1.  , 1.01, 1.01, 0.99, 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.99, 1.  , 1.  ,
       1.  , 1.  , 1.  , 0.99, 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 0.99, 1.  , 1.  , 1.  , 1.  , 0.99,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ])